## 数据集
MovieLens 100k

训练集

测试集

userid | itemid | rating | timestamp

FM用于CTR预估

In [28]:
# QQ音乐数据
import numpy as np
import pandas as pd
import tensorflow as tf
from time import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [29]:
import numpy as np
import pandas as pd
import time
import tensorflow as tf


In [30]:
# cols = ['user','item','rating','timestamp']
# train_df = pd.read_csv('../data/ml-100k/ua.base',delimiter = '\t',names = cols)
# test_df = pd.read_csv('../data/ml-100k/ua.test',delimiter='\t', names= cols)


In [31]:
cols = [str(i) for i in range(10)] # 第一列label，其余列为feature
all_df = pd.read_csv('data/test.txt',header=0,delimiter = ',',names = cols)


feature_df = all_df[cols[1:]]
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scaled_feature = minmax_scaler.fit_transform(feature_df)
x_train,x_test,y_train,y_test = train_test_split(scaled_feature, all_df[cols[0]], train_size = 0.8, random_state = 0)

/home/zjw/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [44]:
feature_size = feature_df.shape[1]
embedding_size = 5

x = tf.placeholder(tf.float32, [None, feature_size] , name='features')
y = tf.placeholder(tf.float32, [None] , name='labels')

embeddings = tf.Variable(tf.random_normal([feature_size, embedding_size], mean=0.0, stddev=0.01), 
                         name='feature_embeddings') # 
feature_value = tf.reshape(tf.cast(x,tf.float32), shape=[-1, feature_size, 1])
w0 = tf.Variable(tf.random_normal([1], mean=0.0, stddev=0.01))

# ------ first-order term ------
feature_bias = tf.Variable(tf.random_uniform([feature_size, 1], 0.0, 1.0), name='feature_bias') # 初始化w_i
y_first_order = feature_bias
y_first_order = tf.reduce_sum(tf.multiply(y_first_order, feature_value), axis=2) # w_i * x_i  (n * k)

# ------ second-order term ------
# 和平方部分
embeddings = tf.multiply(embeddings, feature_value) # v_i * x_i
summed_feature_emb = tf.reduce_sum(embeddings,axis=1)     # n * k
summed_feature_emb_square = tf.square(summed_feature_emb) # n * k 

# 平方和部分
squared_feature_emb = tf.square(embeddings)
squared_sum_feature_emb = tf.reduce_sum(squared_feature_emb, axis = 1)

# 和平方部分与平方和部分组合
y_second_order = 0.5 * tf.subtract(summed_feature_emb_square, squared_sum_feature_emb)

y_predict = tf.add(tf.reduce_sum(y_first_order,1), tf.reduce_sum(y_second_order,1))
y_predict = tf.add(w0, y_predict)
# y_predict = tf.nn.sigmoid(y_predict)

epochs = 200

# loss定义为均方差
loss = tf.reduce_mean(tf.square(y_predict - y))
auc = tf.contrib.metrics.streaming_auc(y_predict,tf.convert_to_tensor(y))  
train = tf.train.AdamOptimizer(learning_rate=0.01,epsilon=1e-8).minimize(loss)

In [45]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(tf.local_variables_initializer())
    print('training...')
    for _ in range(epochs):
        sess.run(train, feed_dict = {x:x_train, y: y_train})
        if _ % 10 == 0:
            print("[%d]loss:%s"%(_,sess.run(loss, feed_dict={x: x_train, y: y_train})))
    
    print('testing...')
    print('MSE:',sess.run(loss, feed_dict={x: x_test ,y: y_test}))
    print('AUC:',sess.run(auc, feed_dict={x: x_test ,y: y_test}))

training...
[0]loss:0.185139
[10]loss:0.179217
[20]loss:0.176569
[30]loss:0.172604
[40]loss:0.169132
[50]loss:0.166963
[60]loss:0.165328
[70]loss:0.163931
[80]loss:0.162557
[90]loss:0.161241
[100]loss:0.160135
[110]loss:0.159292
[120]loss:0.158681
[130]loss:0.158234
[140]loss:0.157884
[150]loss:0.157595
[160]loss:0.157353
[170]loss:0.157149
[180]loss:0.156979
[190]loss:0.156837
testing...
MSE: 0.156734
AUC: (0.0, 0.74744475)
